In [1]:
import matplotlib.pyplot as plt # For plotting
import numpy as np              # Linear algebra library
import pandas as pd

In [2]:
! pwd
expr_df = pd.read_csv("../metadata/length_and_depth.csv")
expr_df = expr_df.drop("nvar", axis=1)
expr_df = expr_df.rename(columns={
    "length" : "expr_length",
    "depth" : "expr_depth"
})
expr_df

'pwd' is not recognized as an internal or external command,
operable program or batch file.


,problem,objtype,variable_nvar,expr_length,expr_depth
0,NZF1,least_squares,True,250,8
1,arglina,least_squares,True,10404,9
2,arglinb,least_squares,True,40200,8
3,arglinc,other,True,39204,7
4,argtrig,other,True,497,7
...,...,...,...,...,...
100,tridia,other,True,398,6
101,vardim,other,True,798,6
102,vibrbeam,least_squares,False,481,13
103,watson,least_squares,False,3900,12


In [3]:
complete_df = pd.read_csv("../results/complete_dataset_as_of_nov6.csv")
complete_df = complete_df[complete_df["is_init_run"] == 0]
complete_df = complete_df.rename(columns={"name" : "problem"})
df = complete_df.merge(
    expr_df,
    on = 'problem',
    how = "inner"
)
df

,status,problem,solver,mem,nvar,time,memory,num_iter,nvmops,neval_obj,...,neval_grad,init_eval_grad_time,init_eval_grad_mem,init_eval_grad_alloc,is_init_run,is_scalable,objtype,variable_nvar,expr_length,expr_depth
0,first_order,NZF1,LBFGSSolver,1,91,1.409402,0.125416,287,287,328,...,315,0.000031,0.001440,5,False,False,least_squares,True,250,8
1,first_order,NZF1,LBFGSSolver,2,91,1.407668,0.095296,199,199,223,...,216,0.000019,0.001440,5,False,False,least_squares,True,250,8
2,first_order,NZF1,LBFGSSolver,3,91,1.407882,0.103456,213,213,234,...,228,0.000016,0.001440,5,False,False,least_squares,True,250,8
3,first_order,NZF1,LBFGSSolver,4,91,1.407476,0.098352,191,191,209,...,203,0.000017,0.001440,5,False,False,least_squares,True,250,8
4,first_order,NZF1,LBFGSSolver,5,91,1.407906,0.108416,209,209,231,...,225,0.000017,0.001440,5,False,False,least_squares,True,250,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18821,first_order,woods,LBFGSSolver,96,1000,1.482090,3.197624,64,64,84,...,75,0.001356,0.008664,5,False,True,other,True,425,7
18822,first_order,woods,LBFGSSolver,97,1000,1.481800,3.229800,64,64,84,...,75,0.001373,0.008664,5,False,True,other,True,425,7
18823,first_order,woods,LBFGSSolver,98,1000,1.482000,3.261976,64,64,84,...,75,0.001367,0.008664,5,False,True,other,True,425,7
18824,first_order,woods,LBFGSSolver,99,1000,1.481908,3.294152,64,64,84,...,75,0.001358,0.008664,5,False,True,other,True,425,7


In [4]:
group_cols = ["problem", "nvar"]

d = df[df["time"].notna()].copy()
d = d.sort_values(group_cols + ["time", "mem"], ascending=True)

best_idx = d.groupby(group_cols)["time"].idxmin()
best_mem_df = d.loc[best_idx].reset_index(drop=True)
best_mem_df

,status,problem,solver,mem,nvar,time,memory,num_iter,nvmops,neval_obj,...,neval_grad,init_eval_grad_time,init_eval_grad_mem,init_eval_grad_alloc,is_init_run,is_scalable,objtype,variable_nvar,expr_length,expr_depth
0,first_order,NZF1,LBFGSSolver,64,91,1.405412,0.241360,59,59,70,...,64,0.000017,0.001440,5,False,False,least_squares,True,250,8
1,first_order,NZF1,LBFGSSolver,72,988,3.127719,2.406504,81,81,94,...,86,0.001715,0.008600,5,False,True,least_squares,True,250,8
2,first_order,arglina,LBFGSSolver,4,100,0.004524,0.029440,1,1,3,...,3,0.000059,0.001520,5,False,False,least_squares,True,10404,9
3,first_order,arglina,LBFGSSolver,5,1000,1.400665,0.240920,1,1,3,...,3,0.005946,0.008664,5,False,True,least_squares,True,10404,9
4,first_order,arglinb,LBFGSSolver,1,100,1.422291,0.020488,2,2,29,...,5,0.000051,0.001520,5,False,False,least_squares,True,40200,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,first_order,vardim,LBFGSSolver,21,1000,28.733923,1.509576,1165,1165,6897,...,6873,0.004314,0.008664,5,False,True,other,True,798,6
180,first_order,vibrbeam,LBFGSSolver,11,8,0.578265,0.054456,103,103,179,...,126,0.000006,0.000720,5,False,False,least_squares,False,481,13
181,unbounded,watson,LBFGSSolver,51,31,1.019050,32.288992,1,1,8,...,8,0.001956,4.349024,42554,False,False,least_squares,False,3900,12
182,first_order,woods,LBFGSSolver,8,100,1.288446,0.060864,45,45,69,...,60,0.000016,0.001520,5,False,False,other,True,425,7


In [15]:
best_mem_map = d.loc[best_idx].set_index(group_cols)["mem"]

df = df.copy()
df["best_problem_mem"] = df.set_index(group_cols).index.map(best_mem_map)
df

,status,problem,solver,mem,nvar,time,memory,num_iter,nvmops,neval_obj,...,init_eval_grad_mem,init_eval_grad_alloc,is_init_run,is_scalable,objtype,variable_nvar,expr_length,expr_depth,is_best_mem,best_problem_mem
0,first_order,NZF1,LBFGSSolver,1,91,1.409402,0.125416,287,287,328,...,0.001440,5,False,False,least_squares,True,250,8,0,64
1,first_order,NZF1,LBFGSSolver,2,91,1.407668,0.095296,199,199,223,...,0.001440,5,False,False,least_squares,True,250,8,0,64
2,first_order,NZF1,LBFGSSolver,3,91,1.407882,0.103456,213,213,234,...,0.001440,5,False,False,least_squares,True,250,8,0,64
3,first_order,NZF1,LBFGSSolver,4,91,1.407476,0.098352,191,191,209,...,0.001440,5,False,False,least_squares,True,250,8,0,64
4,first_order,NZF1,LBFGSSolver,5,91,1.407906,0.108416,209,209,231,...,0.001440,5,False,False,least_squares,True,250,8,0,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18821,first_order,woods,LBFGSSolver,96,1000,1.482090,3.197624,64,64,84,...,0.008664,5,False,True,other,True,425,7,0,8
18822,first_order,woods,LBFGSSolver,97,1000,1.481800,3.229800,64,64,84,...,0.008664,5,False,True,other,True,425,7,0,8
18823,first_order,woods,LBFGSSolver,98,1000,1.482000,3.261976,64,64,84,...,0.008664,5,False,True,other,True,425,7,0,8
18824,first_order,woods,LBFGSSolver,99,1000,1.481908,3.294152,64,64,84,...,0.008664,5,False,True,other,True,425,7,0,8


In [16]:
# unique instances: one row per (problem, nvar)
instances = df[["problem", "nvar"]].drop_duplicates()

# shuffle instances
instances = instances.sample(frac=1, random_state=42).reset_index(drop=True)

n = len(instances)
n_train = int(0.7 * n)
n_valid = int(0.15 * n)

train_inst = instances.iloc[:n_train]
valid_inst = instances.iloc[n_train:n_train + n_valid]
test_inst  = instances.iloc[n_train + n_valid:]

# assign rows to splits by (problem, nvar)
train_df = df.merge(train_inst, on=["problem", "nvar"], how="inner").reset_index(drop=True)
valid_df = df.merge(valid_inst, on=["problem", "nvar"], how="inner").reset_index(drop=True)
test_df  = df.merge(test_inst,  on=["problem", "nvar"], how="inner").reset_index(drop=True)
train_df

,status,problem,solver,mem,nvar,time,memory,num_iter,nvmops,neval_obj,...,init_eval_grad_mem,init_eval_grad_alloc,is_init_run,is_scalable,objtype,variable_nvar,expr_length,expr_depth,is_best_mem,best_problem_mem
0,first_order,NZF1,LBFGSSolver,1,91,1.409402,0.125416,287,287,328,...,0.001440,5,False,False,least_squares,True,250,8,0,64
1,first_order,NZF1,LBFGSSolver,2,91,1.407668,0.095296,199,199,223,...,0.001440,5,False,False,least_squares,True,250,8,0,64
2,first_order,NZF1,LBFGSSolver,3,91,1.407882,0.103456,213,213,234,...,0.001440,5,False,False,least_squares,True,250,8,0,64
3,first_order,NZF1,LBFGSSolver,4,91,1.407476,0.098352,191,191,209,...,0.001440,5,False,False,least_squares,True,250,8,0,64
4,first_order,NZF1,LBFGSSolver,5,91,1.407906,0.108416,209,209,231,...,0.001440,5,False,False,least_squares,True,250,8,0,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13021,first_order,vardim,LBFGSSolver,96,1000,29.012861,3.926536,1165,1165,6897,...,0.008664,5,False,True,other,True,798,6,0,21
13022,first_order,vardim,LBFGSSolver,97,1000,28.999431,3.958712,1165,1165,6897,...,0.008664,5,False,True,other,True,798,6,0,21
13023,first_order,vardim,LBFGSSolver,98,1000,28.868402,3.990888,1165,1165,6897,...,0.008664,5,False,True,other,True,798,6,0,21
13024,first_order,vardim,LBFGSSolver,99,1000,28.898239,4.023064,1165,1165,6897,...,0.008664,5,False,True,other,True,798,6,0,21


In [50]:
from sklearn.ensemble import RandomForestRegressor
feature_cols = ["nvar", 
                "expr_length", 
                "expr_depth", 
                "init_eval_obj_time", 
                "init_eval_grad_time",
                "mem",
                ]
target_col = "time"

X_train = train_df[feature_cols].to_numpy(dtype=float)
X_valid = valid_df[feature_cols].to_numpy(dtype=float)
X_test  = test_df[feature_cols].to_numpy(dtype=float)

t_train = np.log1p(train_df[target_col].to_numpy(dtype=float))
t_valid = np.log1p(valid_df[target_col].to_numpy(dtype=float))
t_test  = np.log1p(test_df[target_col].to_numpy(dtype=float))

In [51]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

best_rf = None
best_score = np.inf
best_params = None

random_state = 66

for n_estimators in range(100, 200, 10):
    for max_depth in [3, 5, 7, 9]:
        for min_leaf in [1, 2, 5]:
            rf = RandomForestRegressor(
                n_estimators=n_estimators,
                max_depth=max_depth,
                min_samples_leaf=min_leaf,
                random_state=66,
                n_jobs=-1,
            )
            rf.fit(X_train, t_train)

            pred_valid = rf.predict(X_valid)
            mse_valid = mean_squared_error(t_valid, pred_valid)

            if mse_valid < best_score:
                best_score = mse_valid
                best_rf = rf
                best_params = (n_estimators, max_depth, min_leaf)
                print(
                        "New best:",
                        "n =", n_estimators,
                        "depth =", max_depth,
                        "min_leaf =", min_leaf,
                        "mse =", mse_valid,
                    )
            else:
                print(
                        "n =", n_estimators,
                        "depth =", max_depth,
                        "min_leaf =", min_leaf,
                        "mse =", mse_valid,
                    )

print("Best validation MSE (forest):", best_score)
print("Best params (n_estimators, max_depth, min_leaf):", best_params)


New best: n = 100 depth = 3 min_leaf = 1 mse = 1.5058664798299481
n = 100 depth = 3 min_leaf = 2 mse = 1.5058664798299481
n = 100 depth = 3 min_leaf = 5 mse = 1.5058664798299481
New best: n = 100 depth = 5 min_leaf = 1 mse = 1.4835847083227278
New best: n = 100 depth = 5 min_leaf = 2 mse = 1.4834936050581524
New best: n = 100 depth = 5 min_leaf = 5 mse = 1.4814392539553352
New best: n = 100 depth = 7 min_leaf = 1 mse = 1.2611928974825002
n = 100 depth = 7 min_leaf = 2 mse = 1.2676632998593615
n = 100 depth = 7 min_leaf = 5 mse = 1.2731203601742362
n = 100 depth = 9 min_leaf = 1 mse = 1.4215184485456969
n = 100 depth = 9 min_leaf = 2 mse = 1.3018370326872735
n = 100 depth = 9 min_leaf = 5 mse = 1.2839351825741274
n = 110 depth = 3 min_leaf = 1 mse = 1.5062970410007133
n = 110 depth = 3 min_leaf = 2 mse = 1.5062970410007133
n = 110 depth = 3 min_leaf = 5 mse = 1.5062970410007133
n = 110 depth = 5 min_leaf = 1 mse = 1.4840775820345502
n = 110 depth = 5 min_leaf = 2 mse = 1.483967773419340

In [56]:
import statsmodels.api as sm
best_n, best_depth, best_min_leaf = best_params

X_train_full = np.vstack([X_train, X_valid])
t_train_full = np.concatenate([t_train, t_valid])
y_train_full = np.log1p(t_train_full)

final_rf = RandomForestRegressor(
    n_estimators=best_n,
    max_depth=best_depth,
    min_samples_leaf=best_min_leaf,
    random_state=0,
    n_jobs=-1,
)
final_rf.fit(X_train_full, y_train_full)

# predict on test in log1p space
pred_test_log = final_rf.predict(X_test)

# convert back to time
pred_test_time = np.expm1(pred_test_log)

test_mse = mean_squared_error(t_test, pred_test_time)
test_mae = mean_absolute_error(t_test, pred_test_time)
test_r2  = r2_score(t_test, pred_test_time)

print("Test MSE (time):", test_mse)
print("Test MAE (time):", test_mae)
print("Test R^2 (time):", test_r2)


ValueError: X has 5 features, but RandomForestRegressor is expecting 6 features as input.

In [57]:
feature_cols = [c for c in feature_cols if c != "mem"]
X_test  = test_df[feature_cols].to_numpy(dtype=float)

def choose_best_mem(model, x_problem, mem_candidates):
    preds = []
    for mem in mem_candidates:
        x = np.concatenate([x_problem, [mem]])
        preds.append((mem, model.predict(x.reshape(1, -1))[0]))
    return min(preds, key=lambda x: x[1])


In [58]:
best_n_estimators, best_max_depth, best_min_leaf = best_params

X_train_full = np.vstack([X_train, X_valid])
t_train_full = np.concatenate([t_train, t_valid])

final_rf = RandomForestRegressor(
    n_estimators=best_n_estimators,
    max_depth=best_max_depth,
    min_samples_leaf=best_min_leaf,
    random_state=0,
    n_jobs=-1,
)
final_rf.fit(X_train_full, t_train_full)

,n_estimators,100
,criterion,'squared_error'
,max_depth,7
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [59]:
mem_candidates = list(range(1, 101))

preds = []
for x_problem in X_test:
    best_mem_pred, pred_time = choose_best_mem(final_rf, x_problem, mem_candidates)
    preds.append(best_mem_pred)
    print(f"Best mem for problem {x_problem} is {best_mem_pred} with predicted time {pred_time}")

preds = np.array(preds)
preds

Best mem for problem [1.0000e+02 1.0404e+04 9.0000e+00 4.7390e-06 5.4454e-05] is 3 with predicted time 0.8608704978385593
Best mem for problem [1.0000e+02 1.0404e+04 9.0000e+00 4.1780e-06 6.5666e-05] is 12 with predicted time 0.8608704978385593
Best mem for problem [1.0000e+02 1.0404e+04 9.0000e+00 4.0480e-06 5.8883e-05] is 58 with predicted time 0.8608704978385593
Best mem for problem [1.0000e+02 1.0404e+04 9.0000e+00 3.9580e-06 5.8813e-05] is 42 with predicted time 0.8608704978385593
Best mem for problem [1.0000e+02 1.0404e+04 9.0000e+00 3.8670e-06 5.8682e-05] is 18 with predicted time 0.8608704978385593
Best mem for problem [1.0000e+02 1.0404e+04 9.0000e+00 3.9680e-06 5.9213e-05] is 57 with predicted time 0.8608704978385593
Best mem for problem [1.0000e+02 1.0404e+04 9.0000e+00 3.7980e-06 6.5496e-05] is 41 with predicted time 0.8602381351202085
Best mem for problem [1.0000e+02 1.0404e+04 9.0000e+00 3.8380e-06 5.8683e-05] is 74 with predicted time 0.8608704978385593
Best mem for prob

KeyboardInterrupt: 

In [60]:
import numpy as np

def choose_best_mem_vectorized(model, x_problem, mem_candidates):
    """
    Given problem features (no mem) and a list/array of candidate mem values,
    return (best_mem, best_predicted_time) by minimizing model-predicted time.
    Assumes the model was trained with features ordered as [problem_features..., mem].
    """
    mem_candidates = np.asarray(mem_candidates, dtype=float)

    X = np.hstack([
        np.repeat(x_problem.reshape(1, -1), len(mem_candidates), axis=0),
        mem_candidates.reshape(-1, 1)
    ])

    y = model.predict(X)
    j = int(np.argmin(y))
    return int(mem_candidates[j]), float(y[j])

mem_candidates = np.arange(1, 101)

preds = []
pred_times = []

for x_problem in X_test:  # X_test must be problem features only (no mem)
    best_mem_pred, pred_time = choose_best_mem_vectorized(final_rf, x_problem, mem_candidates)
    preds.append(best_mem_pred)
    pred_times.append(pred_time)
    print(f"Best mem for problem {x_problem} is {best_mem_pred} with predicted time {pred_time}")

preds = np.array(preds, dtype=int)
pred_times = np.array(pred_times, dtype=float)


Best mem for problem [1.0000e+02 1.0404e+04 9.0000e+00 4.7390e-06 5.4454e-05] is 3 with predicted time 0.8608704978385597
Best mem for problem [1.0000e+02 1.0404e+04 9.0000e+00 4.1780e-06 6.5666e-05] is 3 with predicted time 0.8608704978385598
Best mem for problem [1.0000e+02 1.0404e+04 9.0000e+00 4.0480e-06 5.8883e-05] is 3 with predicted time 0.8608704978385596
Best mem for problem [1.0000e+02 1.0404e+04 9.0000e+00 3.9580e-06 5.8813e-05] is 3 with predicted time 0.8608704978385596
Best mem for problem [1.0000e+02 1.0404e+04 9.0000e+00 3.8670e-06 5.8682e-05] is 3 with predicted time 0.8608704978385597
Best mem for problem [1.0000e+02 1.0404e+04 9.0000e+00 3.9680e-06 5.9213e-05] is 3 with predicted time 0.8608704978385593
Best mem for problem [1.0000e+02 1.0404e+04 9.0000e+00 3.7980e-06 6.5496e-05] is 39 with predicted time 0.8602381351202086
Best mem for problem [1.0000e+02 1.0404e+04 9.0000e+00 3.8380e-06 5.8683e-05] is 3 with predicted time 0.8608704978385596
Best mem for problem [1

In [ ]:
preds
s

array([3, 3, 3, ..., 1, 1, 1], shape=(3000,))

In [ ]:

test_score = np.mean(y_pred_test == X_test["best_problem_mem"])

test_mse = mean_squared_error(t_test, y_pred_test)
test_mae = mean_absolute_error(t_test, y_pred_test)
test_r2  = r2_score(t_test, y_pred_test)

print("Random forest test MSE:", test_mse)
print("Random forest test MAE:", test_mae)
print("Random forest test R^2:", test_r2)

In [ ]:
problem_feature_cols = [c for c in feature_cols if c != "mem"]
mem_candidates_global = np.sort(train_df["mem"].unique())

In [ ]:
def evaluate_mem_selector(model, df_split):
    rows = []

    # treat (problem, nvar) as one instance
    for (prob, nvar), group in df_split.groupby(["problem", "nvar"]):
        mems_available = np.sort(group["mem"].unique())
        mem_candidates = [m for m in mem_candidates_global if m in mems_available]
        if not mem_candidates:
            continue

        # build x_problem without mem
        row0 = group.iloc[0]
        x_problem = row0[problem_feature_cols].to_numpy(dtype=float)

        # this is where you use choose_best_mem
        mem_pred, _ = choose_best_mem(model, x_problem, mem_candidates)

        # true best mem and time for this instance
        best_row = group.loc[group["time"].idxmin()]
        mem_best = best_row["mem"]
        time_best = best_row["time"]

        # actual time if we use mem_pred
        time_chosen = group.loc[group["mem"] == mem_pred, "time"].min()

        rows.append(
            {
                "problem": prob,
                "nvar": nvar,
                "mem_pred": mem_pred,
                "mem_best": mem_best,
                "time_best": time_best,
                "time_chosen": time_chosen,
            }
        )

    res = pd.DataFrame(rows)
    if len(res) == 0:
        return res, float("nan"), float("nan")

    frac_exact = (res["mem_pred"] == res["mem_best"]).mean()
    avg_ratio = (res["time_chosen"] / res["time_best"]).mean()
    return res, frac_exact, avg_ratio


In [ ]:
test_res_rf, frac_exact_rf, avg_ratio_rf = evaluate_mem_selector(final_rf, test_df)
print("forest exact match fraction:", frac_exact_rf)
print("forest avg time_chosen / time_best:", avg_ratio_rf)

In [ ]:
# random forest mem selector evaluation
# test_res_rf, frac_exact_rf, avg_ratio_rf = evaluate_mem_selector(final_rf, test_df)

ratio_rf = test_res_rf["time_chosen"] / test_res_rf["time_best"]

print("RF exact match fraction:", frac_exact_rf)
print("RF avg time_chosen / time_best:", avg_ratio_rf)
print("RF median time_chosen / time_best:", ratio_rf.median())
print("RF frac within 5%:", (ratio_rf <= 1.05).mean())
print("RF frac within 10%:", (ratio_rf <= 1.10).mean())


In [ ]:
rf_tree0 = final_rf.estimators_[0]
from sklearn import tree

plt.figure(figsize=(18, 10))
tree.plot_tree(
    rf_tree0,
    feature_names=feature_cols,
    filled=True,
    rounded=True,
    max_depth=3,
    fontsize=8,
    precision=6,
)
plt.title("Random Forest: tree 0 (top 3 levels)")
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.tree import export_graphviz
import os

dot_data = export_graphviz(
    rf_tree0,
    max_depth=6,
    out_file=None,               # return string instead of writing directly to file
    feature_names=feature_cols,
    filled=True,
    rounded=True,
    precision=6                  # <-- more decimal places
)


In [ ]:
from graphviz import Source

os.makedirs("../tree_plots", exist_ok=True)

tree_graph = Source(dot_data)
pdf_path = tree_graph.render(
    filename="random_forest",    # base name
    directory="../tree_plots",
    format="pdf",
    cleanup=True                     # delete intermediate .dot
)
print("PDF written to:", pdf_path)
tree_graph

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

rf_importances = final_rf.feature_importances_
indices_rf = np.argsort(rf_importances)[::-1]  

plt.figure(figsize=(10, 5))
plt.bar(range(len(feature_cols)), rf_importances[indices_rf])
plt.xticks(range(len(feature_cols)), [feature_cols[i] for i in indices_rf], rotation=45, ha="right")
plt.ylabel("Feature importance")
plt.title("Random forest feature importances")
plt.tight_layout()
plt.show()
